https://docs.autoinfra.cn/docs/integrations/llms/ollama

https://python.langchain.com/docs/use_cases/question_answering/local_retrieval_qa

Tags: langchain, Ollama, Llama2, RAG

In [1]:
from llama_index.llms import Ollama

llm = Ollama(model="llama2:7b-chat-q8_0", request_timeout=60.0)

resp = llm.complete("Who is Paul Graham?")

print(resp)

Paul Graham is a well-known entrepreneur, computer scientist, and writer who has made significant contributions to the fields of artificial intelligence, computer science, and startup culture. He is best known as the co-founder of Y Combinator, a successful start-up accelerator that has launched many successful companies, including Airbnb, Dropbox, and Reddit.

Graham was born in 1964 in Boston, Massachusetts, and grew up in a family of computer scientists. He received his Bachelor's degree in Computer Science from Harvard University in 1982 and his PhD in Computer Science from the University of California, Berkeley in 1987. After completing his PhD, Graham worked as a researcher at various universities and companies, including the University of California, Los Angeles (UCLA) and Bell Labs.

In 1989, Graham co-founded Y Combinator with his wife, Wendy Haisler. Y Combinator is a start-up accelerator that provides funding, mentorship, and resources to early-stage start-ups in exchange fo

In [2]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler                                  
llm = Ollama(base_url="http://localhost:11434", 
             model="llama2:7b-chat-q8_0", 
             callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))
llm("Tell me about the history of AI")

/Users/mjin/Projects/artificical_intelligence/llm/llm_playground/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



 Artificial intelligence (AI) has a rich and varied history that spans several decades. Here is a brief overview of some of the key milestones in the development of AI:

1. 1950s-1960s: The Dartmouth Conference and the Birth of AI
The field of AI was formally established in 1956 at a conference held at Dartmouth College in Hanover, New Hampshire. Attendees included computer scientists, mathematicians, and cognitive scientists who were interested in exploring the possibilities of creating machines that could simulate human intelligence.
2. 1951: The Turing Test
British mathematician Alan Turing proposed a test to measure a machine's ability to exhibit intelligent behavior equivalent to, or indistinguishable from, that of a human. The Turing Test has since become a benchmark for measuring the success of AI systems.
3. 1956: The First AI Program
John McCarthy, a computer scientist at Stanford University, created the first AI program, called the Logical Theorist, which was designed to rea

'\n Artificial intelligence (AI) has a rich and varied history that spans several decades. Here is a brief overview of some of the key milestones in the development of AI:\n\n1. 1950s-1960s: The Dartmouth Conference and the Birth of AI\nThe field of AI was formally established in 1956 at a conference held at Dartmouth College in Hanover, New Hampshire. Attendees included computer scientists, mathematicians, and cognitive scientists who were interested in exploring the possibilities of creating machines that could simulate human intelligence.\n2. 1951: The Turing Test\nBritish mathematician Alan Turing proposed a test to measure a machine\'s ability to exhibit intelligent behavior equivalent to, or indistinguishable from, that of a human. The Turing Test has since become a benchmark for measuring the success of AI systems.\n3. 1956: The First AI Program\nJohn McCarthy, a computer scientist at Stanford University, created the first AI program, called the Logical Theorist, which was desig

In [3]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [5]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

Document(page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"})

In [6]:
from langchain import PromptTemplate

# 提示
template = """Use the following context to answer the last question.
If you don't know the answer, just say 'I don't know', don't try to make up an answer.
Answer the question briefly with maximun three sentences.
{context}
Question: {question}
Useful Answer: """
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [7]:
# LLM
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Ollama(base_url="http://localhost:11434",
             model="llama2:7b-chat-q8_0",
             verbose=True,
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

In [8]:
# QA链
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [9]:
question = "What are the approaches to AI Agent Task Decomposition?"
result = qa_chain({"query": question})

/Users/mjin/Projects/artificical_intelligence/llm/llm_playground/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The approaches to AI agent task decomposition include:

1. Using large language models (LLM) with simple prompting, such as "Steps for XYZ." or "What are the subgoals for achieving XYZ?"
2. Using task-specific instructions, such as "Write a story outline" for writing a novel.
3. Using human inputs to guide the decomposition process.

In [10]:
62 / (1313002000/1000/1000/1000)

47.22003469910937